## Simulations Correlated ##
Simulations for correlated errors 

In [ ]:
# qiskit does not play nicely with modern numpy
import warnings

warnings.catch_warnings() 
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# Mainline Python Imports
import random
import math
import numpy as np
import copy
import sys, os, time

from functools import reduce


# Plotting Imports
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sbs

sbs.set(style="darkgrid")
%matplotlib inline

from qutip import *
import scipy

# Qiskit Imports
import qiskit
from qiskit import IBMQ
from qiskit import transpile, QuantumRegister, assemble
from qiskit import QuantumCircuit, execute, Aer, QuantumCircuit
from qiskit.ignis.mitigation.measurement import complete_meas_cal, tensored_meas_cal, CompleteMeasFitter, TensoredMeasFitter
import qiskit.ignis.verification.randomized_benchmarking as rb

import qiskit
import numpy as np


from PatchedMeasCal import bv


from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal.state_prep_circuits import integer_state_prep_sim, integer_state_dist
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal import utils

from PatchedMeasCal.utils import Progressbar, list_fold
from PatchedMeasCal.utils import norm_results_dict

from PatchedMeasCal.fake_backends import LocalSimulator, Grid
from PatchedMeasCal.gen_error_matrices import FakeMeasurementError


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import mthree

In [ ]:
n_qubits = 4
backend = LocalSimulator(n_qubits)

Construct the error probabilities

In [ ]:
from PatchedMeasCal.gen_error_matrices import FakeMeasurementError

n_qubits = 4
err = 10
# Fixed coupling map for this one
coupling_map = [[0, 1], [1, 2], [2, 3]]
probs = FakeMeasurementError(
        [100, 0, err, 0, 0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0], # 0 -> 1 - Controls error biases
        n_qubits = n_qubits, 
        coupling_map = coupling_map
)


## With Coupling Map

In [ ]:
vals = {
    'Bare':[],
    'AIM':[],
    'SIM':[],
    'JIGSAW':[],
    'JIGSAW-FIX':[],
    'Full':[],
    'Linear':[],
    'CMC':[],
    'CMC-2':[],
    'CMC-3':[],
    'CMC-5':[],
    'CMC-JIGSAW':[],
    'JIGSAW-CMC':[],
    'MTHREE':[],
    'idx':[]
}

err_range = (10, 11)
shots_range = (1000, 16001, 1000)


pb = Progressbar(20,  len(list(range(*shots_range))) * len(list(range(*err_range))) * (n_qubits ** 2))
for err in range(*err_range):
    
    for n_shots in range(*shots_range):
        coupling_map = list(range(n_qubits))
        random.shuffle(coupling_map)
        coupling_map = [[coupling_map[i], coupling_map[i + 1]] for i in range(len(coupling_map) - 1)]

        probs = FakeMeasurementError(
            [100, 0, err, 0, 0], # Const - Controls correlation of error weights
            [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
            [0, 0, 0, 0, 0], # 0 -> 1 - Controls error biases
            n_qubits = n_qubits, 
            coupling_map = coupling_map
            )
    
        n_shots_build = n_shots // 2
        n_shots_bare = n_shots

        tpf = TensorPatchFitter(backend, coupling_map=coupling_map)
        tpf.build(probs=probs, n_shots=n_shots_build)
        
        probs_filtered = copy.deepcopy(probs)
        probs_filtered.meas_filter = tpf
        
        q_full = qiskit_full(backend, n_qubits, n_shots, probs=probs)
        q_lin = qiskit_linear(backend, n_qubits, n_shots, probs=probs)
        
        mit = mthree.M3Mitigation(backend, probs=probs)
        mit.cals_from_system(list(range(n_qubits)), n_shots_build)
        
        
        for i in range(n_qubits ** 2):
            pb.tick()
            circuit = integer_state_prep_sim(i, n_qubits)

            bare_val = qiskit.execute(circuit, backend, shots=n_shots).result().get_counts()
            bare_val = probs(bare_val)

            vals['Full'].append(1 - integer_state_dist(q_full.apply(bare_val), i))
            vals['Linear'].append(1 - integer_state_dist(q_lin.apply(bare_val), i))

            vals['Bare'].append(1 - integer_state_dist(bare_val, i))
            vals['JIGSAW'].append(1 - integer_state_dist(jigsaw(circuit, backend, n_shots_bare, norm_fix=True, probs=probs, n_qubits=n_qubits, equal_shot_distribution = True), i))
            vals['JIGSAW-FIX'].append(1 - integer_state_dist(jigsaw(circuit, backend, n_shots_bare, probs=probs, n_qubits=n_qubits, equal_shot_distribution = True, norm_fix=True), i))
            
            
            vals['AIM'].append(1 - integer_state_dist(aim(circuit, backend, n_qubits, n_shots=n_shots_bare, probs=probs, equal_shot_distribution = True), i))
            vals['SIM'].append(1 - integer_state_dist(sim(circuit, backend, n_qubits, n_shots=n_shots_bare, probs=probs, equal_shot_distribution = True), i))
            vals['CMC'].append(1 - integer_state_dist(tpf.apply(bare_val), i))
            
            vals['CMC-2'].append(1 - integer_state_dist(tpf.apply(tpf.apply(bare_val)), i))
            vals['CMC-3'].append(1 - integer_state_dist(tpf.apply(tpf.apply(tpf.apply(bare_val))), i))
            vals['CMC-5'].append(1 - integer_state_dist(tpf.apply(tpf.apply(tpf.apply(tpf.apply(tpf.apply(bare_val))))), i))

            vals['JIGSAW-CMC'].append(1 - integer_state_dist(
                tpf.apply(
                    jigsaw(circuit, backend, n_shots_bare, probs=probs, n_qubits=n_qubits, equal_shot_distribution = True)
                ), i))
            
            vals['CMC-JIGSAW'].append(1 - integer_state_dist(
                jigsaw(circuit, backend, n_shots_bare, probs=probs_filtered, n_qubits=n_qubits, equal_shot_distribution = True)
                , i))
            
            vals['MTHREE'].append(1 - integer_state_dist(
                mit.apply_correction(bare_val, list(range(n_qubits))).nearest_probability_distribution() 
                , i))
            
            
            vals['idx'].append(n_shots)

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
plt.rcParams["figure.figsize"] = (8, 5.5)
plt.rc('axes', labelsize=14)

fig, axs = plt.subplots(1, 8)
subfig_i = 0

val_keys = ['Bare', 'SIM', 'AIM', 'JIGSAW', 'CMC', 'Linear', 'Full', 'MTHREE']

plt.rc('ytick', labelsize=14)

for key in val_keys:
    if key == 'Linear':
        ax_1 = sbs.histplot(y=vals[key], ax=axs[subfig_i], element='step', bins=14)
    elif 'JIGSAW' in key:
        ax_1 = sbs.histplot(y=vals[key], ax=axs[subfig_i], element='step', bins=60)
    else:
        ax_1 = sbs.histplot(y=vals[key], ax=axs[subfig_i], element='step')
    ax_1.set_ylim([0, 1.02])
    ax_1.set(xlabel=key)
    ax_1.set(xticklabels=[])
    ax_1.plot(ax_1.get_xlim(), [np.mean(vals[key])] * 2, color='black', linewidth=1, linestyle = '--')
    
    if subfig_i == 0:
        ax_1.set(ylabel="Success Probability")
    
    if (subfig_i > 0 and subfig_i != 7 ):
        ax_1.set(ylabel=None, yticklabels=[])
    if (subfig_i == 7):
        ax_1.yaxis.tick_right()
        

    subfig_i += 1

fig.suptitle('Correlated Measurement Error Mitigation', y=0.94, fontsize=14)
#plt.savefig('corr_errors.pdf')